`save_inception_v3.ipynb` で保存したモデルをロードして，推論をやってみる

In [5]:
import numpy as np
import onnxruntime as ort

# モデルをONNX形式で読み出す
ort_session = ort.InferenceSession("./models/inception_v3.onnx")

# ダミー入力の推論を実行
outputs = ort_session.run(
    None,
    {"actual_input_1": np.random.randn(1, 3, 299, 299).astype(np.float32)},
)

# 出力の形状等を確認
print(type(outputs)) # list
print(len(outputs))  # 1

print(type(outputs[0])) # <class 'numpy.ndarray'>
print(outputs[0].shape) # (1, 1000)

print(type(outputs[0][0])) # <class 'numpy.ndarray'>
print(outputs[0][0].shape) # (1000,)

print(outputs[0])

<class 'list'>
1
<class 'numpy.ndarray'>
(1, 1000)
<class 'numpy.ndarray'>
(1000,)
[[-1.17838264e+00  1.06814504e+00  1.89881945e+00  4.49991226e-01
   1.07987547e+00 -3.47007781e-01  5.05505502e-01  1.93027079e+00
   1.30537045e+00  7.30804145e-01  2.22678757e+00  3.14485693e+00
   3.54335022e+00  2.62518167e+00  2.55788159e+00  2.87316513e+00
   3.31987667e+00  1.34876871e+00  3.15137815e+00  2.51203203e+00
   1.06617582e+00  5.27619505e+00  3.65874624e+00  3.04250550e+00
   2.69145298e+00 -5.78665972e-01 -2.78924793e-01 -6.96162283e-01
  -8.72934580e-01 -5.85845113e-01 -3.61756384e-01  9.25235629e-01
  -1.48478997e+00  7.24934220e-01  3.88826936e-01 -1.40951324e+00
   3.59538227e-01 -1.15761292e+00  2.49602303e-01 -1.44783527e-01
   1.41780317e+00 -4.29603189e-01  1.86300218e+00  8.49784672e-01
   2.35189587e-01 -2.42598757e-01  1.46185184e+00  5.47605217e-01
  -9.26724792e-01 -3.98080766e-01  6.71643555e-01 -9.23508346e-01
   5.07237315e-01  1.35562420e+00  1.19030088e-01  6.540867

推論結果のラベルを得るためにImageNetのラベルを取得する

In [7]:
# Download ImageNet labels
!curl -o ./data/imagenet_classes.txt https://raw.githubusercontent.com/pytorch/hub/master/imagenet_classes.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 10472  100 10472    0     0  28769      0 --:--:-- --:--:-- --:--:-- 28769


pytorchのURLから画像を取得して，その画像を推論してみる．


In [23]:
import urllib
url, filename = ("https://github.com/pytorch/hub/raw/master/images/dog.jpg", "./data/dog.jpg")
try: urllib.URLopener().retrieve(url, filename)
except: urllib.request.urlretrieve(url, filename)

In [16]:
# 犬以外にしたい場合は以下をコメントアウト
# filename = './data/siamese_cat.jpg'
# filename = './data/cat.jpg'

In [26]:
# sample execution (requires torchvision)
from PIL import Image
from torchvision import transforms
import torch
# 画像をPILのImage形式でオープン
input_image = Image.open(filename)
# 前処理の定義
preprocess = transforms.Compose([
    transforms.Resize(299),
    transforms.CenterCrop(299),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
# 前処理にかける
input_tensor = preprocess(input_image)
# モデルに入力するためにバッチ次元を増やす
input_batch = input_tensor.unsqueeze(0) # create a mini-batch as expected by the model
print(input_batch.shape)

# モデルの出力を得る(softmax通ってない)
outputs = ort_session.run(
    None,
    {"actual_input_1": input_batch.numpy()},
)

# 推論結果を得る(softmaxに通して確率にする)
probabilities = torch.nn.functional.softmax(torch.from_numpy(outputs[0][0]), dim=0)
print(probabilities.max())
print(probabilities.argmax())

torch.Size([1, 3, 299, 299])
tensor(0.8238)
tensor(258)


In [27]:
# Read the categories
with open("./data/imagenet_classes.txt", "r") as f:
    categories = [s.strip() for s in f.readlines()]
# Show top categories per image
top5_prob, top5_catid = torch.topk(probabilities, 5)
for i in range(top5_prob.size(0)):
    print(categories[top5_catid[i]], top5_prob[i].item())

Samoyed 0.8238480687141418
Arctic fox 0.014191577211022377
white wolf 0.012303497642278671
Pomeranian 0.007874670438468456
keeshond 0.006387793458998203
